In [1]:
import pandas as pd
import sqlite3

#read file
acoustic_df = pd.read_table('msd_genre_dataset.txt', sep = ',')
acoustic_df.head()

,genre,track_id,artist_name,title,loudness,tempo,time_signature,key,mode,duration,...,var_timbre3,var_timbre4,var_timbre5,var_timbre6,var_timbre7,var_timbre8,var_timbre9,var_timbre10,var_timbre11,var_timbre12
0,classic pop and rock,TRFCOOU128F427AEC0,Blue Oyster Cult,Mes Dames Sarat,-8.697,155.007,1,9,1,246.33424,...,1255.514569,580.030472,598.485223,575.337671,322.068603,321.726029,232.700609,186.805303,181.938688,151.508011
1,classic pop and rock,TRNJTPB128F427AE9F,Blue Oyster Cult,Screams,-10.659,148.462,1,4,0,189.80526,...,2007.653070,1043.474073,585.694981,564.013736,510.177022,400.200186,365.119588,238.099708,197.933757,251.577525
2,classic pop and rock,TRLFJHA128F427AEEA,Blue Oyster Cult,Dance The Night Away,-13.494,112.909,1,10,0,158.19710,...,1204.856777,2736.520024,730.233239,665.203452,535.775111,439.335059,486.822970,265.333860,447.097987,251.880724
3,classic pop and rock,TRCQZAG128F427DB97,Blue Oyster Cult,Debbie Denise,-12.786,117.429,4,7,1,250.22649,...,809.755802,563.908070,492.803819,378.382799,372.875044,231.941957,246.313305,168.400152,85.282462,339.897173
4,classic pop and rock,TRNXMNM128F427DB8C,Blue Oyster Cult,(Don't Fear) The Reaper,-14.093,141.536,4,9,0,307.06893,...,1093.684935,343.556047,889.163314,218.111796,304.862864,178.352161,440.478867,142.669283,81.061326,208.355152


In [2]:
selected_columns = [u'track_id', u'loudness', u'tempo',
       u'time_signature', u'key', u'mode', u'duration', u'avg_timbre1',
       u'avg_timbre2', u'avg_timbre3', u'avg_timbre4', u'avg_timbre5',
       u'avg_timbre6', u'avg_timbre7', u'avg_timbre8', u'avg_timbre9',
       u'avg_timbre10', u'avg_timbre11', u'avg_timbre12', u'var_timbre1',
       u'var_timbre2', u'var_timbre3', u'var_timbre4', u'var_timbre5',
       u'var_timbre6', u'var_timbre7', u'var_timbre8', u'var_timbre9',
       u'var_timbre10', u'var_timbre11', u'var_timbre12']
df_selcol = acoustic_df[selected_columns]
df_selcol.columns

Index([u'track_id', u'loudness', u'tempo', u'time_signature', u'key', u'mode',
       u'duration', u'avg_timbre1', u'avg_timbre2', u'avg_timbre3',
       u'avg_timbre4', u'avg_timbre5', u'avg_timbre6', u'avg_timbre7',
       u'avg_timbre8', u'avg_timbre9', u'avg_timbre10', u'avg_timbre11',
       u'avg_timbre12', u'var_timbre1', u'var_timbre2', u'var_timbre3',
       u'var_timbre4', u'var_timbre5', u'var_timbre6', u'var_timbre7',
       u'var_timbre8', u'var_timbre9', u'var_timbre10', u'var_timbre11',
       u'var_timbre12'],
      dtype='object')

In [3]:
df_selcol.head()

,track_id,loudness,tempo,time_signature,key,mode,duration,avg_timbre1,avg_timbre2,avg_timbre3,...,var_timbre3,var_timbre4,var_timbre5,var_timbre6,var_timbre7,var_timbre8,var_timbre9,var_timbre10,var_timbre11,var_timbre12
0,TRFCOOU128F427AEC0,-8.697,155.007,1,9,1,246.33424,46.673067,14.613684,14.664215,...,1255.514569,580.030472,598.485223,575.337671,322.068603,321.726029,232.700609,186.805303,181.938688,151.508011
1,TRNJTPB128F427AE9F,-10.659,148.462,1,4,0,189.80526,43.645377,-87.337150,41.051582,...,2007.653070,1043.474073,585.694981,564.013736,510.177022,400.200186,365.119588,238.099708,197.933757,251.577525
2,TRLFJHA128F427AEEA,-13.494,112.909,1,10,0,158.19710,37.573538,-61.202030,28.760533,...,1204.856777,2736.520024,730.233239,665.203452,535.775111,439.335059,486.822970,265.333860,447.097987,251.880724
3,TRCQZAG128F427DB97,-12.786,117.429,4,7,1,250.22649,42.566615,17.217897,53.484213,...,809.755802,563.908070,492.803819,378.382799,372.875044,231.941957,246.313305,168.400152,85.282462,339.897173
4,TRNXMNM128F427DB8C,-14.093,141.536,4,9,0,307.06893,40.976507,-13.609779,39.566169,...,1093.684935,343.556047,889.163314,218.111796,304.862864,178.352161,440.478867,142.669283,81.061326,208.355152


Insert into SQL database

In [4]:
#Function taken from http://yznotes.com/write-pandas-dataframe-to-sqlite/
def df2sqlite(dataframe, db_name, tbl_name):
    conn=sqlite3.connect(db_name)
    conn.text_factory = lambda x: unicode(x, "utf-8", "ignore")    
    cur = conn.cursor()                                 
 
    wildcards = ','.join(['?'] * len(dataframe.columns))    
    data = [tuple(x) for x in dataframe.values]

    cur.execute("drop table if exists %s" % tbl_name)
 
    col_str = '"' + '","'.join(dataframe.columns) + '"'
    cur.execute("create table %s (%s)" % (tbl_name, col_str))
 
    cur.executemany("insert into %s values(%s)" % (tbl_name, wildcards), data)
 
    conn.commit()
    conn.close()

In [5]:
_DBName = '../../MillionSongSubset/lastfm_tags.db'
_TableName = u'track_acoustic'

In [6]:
df2sqlite(df_selcol, db_name = _DBName, tbl_name = _TableName)

In [7]:
# See the tables
conn=sqlite3.connect(_DBName)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[(u'track_acoustic',)]


In [8]:
# Confirm that the data is there
df = pd.read_sql_query("SELECT * FROM track_acoustic LIMIT 5", conn)
df.head()

,track_id,loudness,tempo,time_signature,key,mode,duration,avg_timbre1,avg_timbre2,avg_timbre3,...,var_timbre3,var_timbre4,var_timbre5,var_timbre6,var_timbre7,var_timbre8,var_timbre9,var_timbre10,var_timbre11,var_timbre12
0,TRFCOOU128F427AEC0,-8.697,155.007,1,9,1,246.33424,46.673067,14.613684,14.664215,...,1255.514569,580.030472,598.485223,575.337671,322.068603,321.726029,232.700609,186.805303,181.938688,151.508011
1,TRNJTPB128F427AE9F,-10.659,148.462,1,4,0,189.80526,43.645377,-87.337150,41.051582,...,2007.653070,1043.474073,585.694981,564.013736,510.177022,400.200186,365.119588,238.099708,197.933757,251.577525
2,TRLFJHA128F427AEEA,-13.494,112.909,1,10,0,158.19710,37.573538,-61.202030,28.760533,...,1204.856777,2736.520024,730.233239,665.203452,535.775111,439.335059,486.822970,265.333860,447.097987,251.880724
3,TRCQZAG128F427DB97,-12.786,117.429,4,7,1,250.22649,42.566615,17.217897,53.484213,...,809.755802,563.908070,492.803819,378.382799,372.875044,231.941957,246.313305,168.400152,85.282462,339.897173
4,TRNXMNM128F427DB8C,-14.093,141.536,4,9,0,307.06893,40.976507,-13.609779,39.566169,...,1093.684935,343.556047,889.163314,218.111796,304.862864,178.352161,440.478867,142.669283,81.061326,208.355152


In [9]:
#Number of rows
df = pd.read_sql_query("SELECT COUNT(*) FROM track_acoustic", conn)
-df.head()

,COUNT(*)
0,-59600


In [10]:
join_statement = 'tid_tag JOIN tids ON (tid_tag.tid=tids.ROWID) JOIN \
tags ON (tags.ROWID=tid_tag.tag) JOIN track_year ON (tids.tid = track_year.tid) JOIN track_acoustic ta ON(tids.tid = ta.track_id)'

drop_view = "DROP VIEW genre_view;"

cursor.execute(drop_view)

query_genres = "CREATE VIEW genre_view AS \
SELECT CASE \
WHEN LOWER(tags.tag) LIKE '%rock%' THEN 'rock' \
WHEN LOWER(tags.tag) LIKE '%pop%' THEN 'pop' \
WHEN LOWER(tags.tag) LIKE '%country%' THEN 'country' \
WHEN LOWER(tags.tag) LIKE '%jazz%' THEN 'jazz' \
WHEN LOWER(tags.tag) LIKE '%blues%' THEN 'blues' \
WHEN LOWER(tags.tag) LIKE '%electronic%' THEN 'electronic' \
WHEN LOWER(tags.tag) LIKE '%indie%' THEN 'indie' \
WHEN LOWER(tags.tag) LIKE '%hip-hop%' THEN 'hip-hop' \
WHEN LOWER(tags.tag) LIKE '%hiphop%' THEN 'hip-hop' \
WHEN LOWER(tags.tag) LIKE '%rap%' THEN 'rap' \
END AS Genre, \
tids.tid AS tid, Year, \
loudness, \
tempo, \
duration, \
avg_timbre1, \
avg_timbre2, \
avg_timbre3, \
avg_timbre4, \
avg_timbre5, \
avg_timbre6, \
avg_timbre7, \
avg_timbre8, \
avg_timbre9, \
avg_timbre10, \
avg_timbre11, \
avg_timbre12, \
var_timbre1, \
var_timbre2, \
var_timbre3, \
var_timbre4, \
var_timbre5, \
var_timbre6, \
var_timbre7, \
var_timbre8, \
var_timbre9, \
var_timbre10, \
var_timbre11, \
var_timbre12 \
FROM {0}\
WHERE LOWER(tags.tag) LIKE '%rock%' OR LOWER(tags.tag) LIKE '%pop%' OR LOWER(tags.tag) LIKE '%country%' \
OR LOWER(tags.tag) LIKE '%jazz%' OR LOWER(tags.tag) LIKE '%blues%' OR LOWER(tags.tag) LIKE '%electronic%' \
OR LOWER(tags.tag) LIKE '%indie%' OR LOWER(tags.tag) LIKE '%hip-hop%' OR LOWER(tags.tag) LIKE '%hiphop%' OR LOWER(tags.tag) LIKE '%rap%';".format(join_statement)

cursor.execute(query_genres)

query = "SELECT Year, COUNT(DISTINCT Genre) FROM genre_view GROUP BY Year"
df = pd.read_sql_query(query, conn)
df

OperationalError: no such view: genre_view

In [12]:
query = "SELECT DISTINCT * FROM genre_view LIMIT 5"
df = pd.read_sql_query(query, conn)
df

DatabaseError: Execution failed on sql 'SELECT DISTINCT * FROM genre_view LIMIT 5': no such table: genre_view

In [14]:
minYear = 1954
maxYear = 2010
acoustic_dict = {}

for year in range(minYear,maxYear+1):
    print(year)
    query_acoustic = "SELECT LOWER(g.Genre) AS Genre, \
AVG(loudness) AS AVG_LOUDNESS, \
AVG(tempo) AS AVG_TEMPO, \
AVG(duration) AS AVG_DURATION, \
AVG(avg_timbre1) AS AVG_TIMBRE1, \
AVG(avg_timbre2) AS AVG_TIMBRE2, \
AVG(avg_timbre3) AS AVG_TIMBRE3, \
AVG(avg_timbre4) AS AVG_TIMBRE4, \
AVG(avg_timbre5) AS AVG_TIMBRE5, \
AVG(avg_timbre6) AS AVG_TIMBRE6, \
AVG(avg_timbre7) AS AVG_TIMBRE7, \
AVG(avg_timbre8) AS AVG_TIMBRE8, \
AVG(avg_timbre9) AS AVG_TIMBRE9, \
AVG(avg_timbre10) AS AVG_TIMBRE10, \
AVG(avg_timbre11) AS AVG_TIMBRE11, \
AVG(avg_timbre12) AS AVG_TIMBRE12, \
AVG(var_timbre1) AS AVG_VAR_TIMBRE1, \
AVG(var_timbre2) AS AVG_VAR_TIMBRE2, \
AVG(var_timbre3) AS AVG_VAR_TIMBRE3, \
AVG(var_timbre4) AS AVG_VAR_TIMBRE4, \
AVG(var_timbre5) AS AVG_VAR_TIMBRE5, \
AVG(var_timbre6) AS AVG_VAR_TIMBRE6, \
AVG(var_timbre7) AS AVG_VAR_TIMBRE7, \
AVG(var_timbre8) AS AVG_VAR_TIMBRE8, \
AVG(var_timbre9) AS AVG_VAR_TIMBRE9, \
AVG(var_timbre10) AS AVG_VAR_TIMBRE10, \
AVG(var_timbre11) AS AVG_VAR_TIMBRE11, \
AVG(var_timbre12) AS AVG_VAR_TIMBRE12 \
FROM {0} g \
WHERE g.Year = {1} \
GROUP BY g.Genre ORDER BY Genre;".format('genre_view', str(year))
    df = pd.read_sql_query(query_acoustic, conn)
    acoustic_dict[year] = df

1954


DatabaseError: Execution failed on sql 'SELECT LOWER(g.Genre) AS Genre, AVG(loudness) AS AVG_LOUDNESS, AVG(tempo) AS AVG_TEMPO, AVG(duration) AS AVG_DURATION, AVG(avg_timbre1) AS AVG_TIMBRE1, AVG(avg_timbre2) AS AVG_TIMBRE2, AVG(avg_timbre3) AS AVG_TIMBRE3, AVG(avg_timbre4) AS AVG_TIMBRE4, AVG(avg_timbre5) AS AVG_TIMBRE5, AVG(avg_timbre6) AS AVG_TIMBRE6, AVG(avg_timbre7) AS AVG_TIMBRE7, AVG(avg_timbre8) AS AVG_TIMBRE8, AVG(avg_timbre9) AS AVG_TIMBRE9, AVG(avg_timbre10) AS AVG_TIMBRE10, AVG(avg_timbre11) AS AVG_TIMBRE11, AVG(avg_timbre12) AS AVG_TIMBRE12, AVG(var_timbre1) AS AVG_VAR_TIMBRE1, AVG(var_timbre2) AS AVG_VAR_TIMBRE2, AVG(var_timbre3) AS AVG_VAR_TIMBRE3, AVG(var_timbre4) AS AVG_VAR_TIMBRE4, AVG(var_timbre5) AS AVG_VAR_TIMBRE5, AVG(var_timbre6) AS AVG_VAR_TIMBRE6, AVG(var_timbre7) AS AVG_VAR_TIMBRE7, AVG(var_timbre8) AS AVG_VAR_TIMBRE8, AVG(var_timbre9) AS AVG_VAR_TIMBRE9, AVG(var_timbre10) AS AVG_VAR_TIMBRE10, AVG(var_timbre11) AS AVG_VAR_TIMBRE11, AVG(var_timbre12) AS AVG_VAR

In [13]:
import pickle
input_file = open('acoustic_dict.dat','rb')

acoustic_dict_noRE = pickle.load(input_file) 
input_file.close()
acoustic_dict_noRE[1999]

,Genre,AVG_LOUDNESS,AVG_TEMPO,AVG_DURATION,AVG_TIMBRE1,AVG_TIMBRE2,AVG_TIMBRE3,AVG_TIMBRE4,AVG_TIMBRE5,AVG_TIMBRE6,...,AVG_VAR_TIMBRE3,AVG_VAR_TIMBRE4,AVG_VAR_TIMBRE5,AVG_VAR_TIMBRE6,AVG_VAR_TIMBRE7,AVG_VAR_TIMBRE8,AVG_VAR_TIMBRE9,AVG_VAR_TIMBRE10,AVG_VAR_TIMBRE11,AVG_VAR_TIMBRE12
0,blues,-8.536417,119.523396,262.112744,44.041691,0.401681,11.009379,2.081100,-9.162332,-3.058484,...,2384.439677,1639.160088,892.792019,970.211447,642.171116,477.040193,422.780006,313.685329,335.567013,354.886968
1,country,-12.412387,115.097871,226.392701,40.594720,-44.659840,21.530288,-2.991239,-2.053335,-2.342639,...,1769.106161,1529.122695,1017.434379,903.171172,676.257396,465.128735,443.823429,321.710380,306.652013,335.185058
2,electronic,-9.746184,123.947612,302.923461,41.867505,-4.229137,-18.504186,7.360948,3.300601,-11.772693,...,2526.279955,2006.899985,1071.228611,1121.187481,702.692020,690.573764,384.798984,390.253280,326.210763,292.131543
3,hip-hop,-7.589750,108.617167,241.257777,43.782269,16.956211,-24.066006,8.423069,5.028266,9.980125,...,2839.853357,3432.671030,1076.338334,1976.842585,963.453462,977.771057,453.827148,589.860168,560.367634,341.753247
4,indie,-8.324414,122.513121,245.229891,44.620585,-8.373739,0.018415,-1.761145,-6.790704,-11.407960,...,1894.010035,1477.563743,914.212255,792.986805,619.314359,570.301371,350.895033,343.910431,264.625147,261.037180
5,jazz,-11.921600,125.213720,353.335971,39.587756,-30.261151,-9.512966,1.364981,6.904706,-10.564618,...,2999.810429,1720.353163,1116.371541,1043.284736,816.685430,554.170118,489.920617,352.309021,300.529497,391.837711
6,pop,-8.428549,125.189632,263.709400,44.154549,-2.322248,-1.725728,-0.704076,2.094191,-11.025260,...,2093.455601,1613.855956,893.628649,885.583289,631.660860,588.439612,365.093477,341.855053,288.955850,293.736583
7,rap,-7.164818,132.248545,309.716800,44.538130,-16.291426,-3.546023,6.953292,4.505882,-8.066438,...,2406.014001,1905.416882,833.655283,1178.033612,771.595317,696.974340,364.084825,411.910364,353.837003,313.488015
8,rock,-7.785214,125.709745,254.101179,45.249337,1.223254,6.858183,0.337822,-8.188387,-9.959818,...,2005.759854,1516.970649,916.950703,844.151861,602.289730,529.336973,362.920914,329.359790,279.220951,281.302909


In [ ]:
acoustic_dict[1960]

In [11]:
#Pickle the pivot dictionary
import pickle
output_file = open('acoustic_dict_complete.dat','wb')
pickle.dump(acoustic_dict, output_file)
output_file.close()

NameError: name 'acoustic_dict' is not defined